# Setup

In [1]:
# Standard library imports
import warnings
import os

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datasetsforecast.hierarchical import HierarchicalData
from hierarchicalforecast.utils import aggregate
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import  BottomUp, TopDown, MiddleOut, MinTrace, ERM
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Configuration & Settings
warnings.simplefilter(action='ignore', category=FutureWarning)

/Users/kbanachewicz/Documents/time-series-with-Konrad/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


This section of code imports necessary libraries for time series forecasting, specifically focusing on hierarchical forecasting. 

First, it brings in modules from Python’s standard library: `warnings` to manage warning messages and `os` for operating system interactions.

Next, several third-party packages are imported. `numpy` provides support for numerical operations, especially with arrays. `pandas` is used for data manipulation and analysis through DataFrames. `matplotlib.pyplot` enables the creation of visualizations like plots and charts. 

The code then imports specific classes and functions from specialized libraries: `datasetsforecast` for handling hierarchical datasets; `hierarchicalforecast` which contains tools for reconciliation, different forecasting methods (BottomUp, TopDown, MiddleOut, MinTrace, ERM), evaluation metrics, and utility functions like aggregation; `statsforecast` provides a framework for statistical time series forecasting with models such as AutoARIMA and Naive; and finally, `sklearn.metrics` offers standard regression metrics like mean absolute error and mean squared error.

The final line suppresses `FutureWarning` messages, which can clutter the output without necessarily indicating critical errors. This is done to keep the console cleaner during execution.

In [2]:
# general settings
class CFG:
    data_folder = './data/'
    img_dim1 = 20
    img_dim2 = 10
    SEED = 42


# display style 
plt.style.use("seaborn-v0_8")
plt.rcParams["figure.figsize"] = (CFG.img_dim1, CFG.img_dim2)

np.random.seed(CFG.SEED)

This code establishes general settings and configures the display style for visualizations. 

A class named `CFG` is defined to hold configuration parameters. It currently defines three attributes: `data_folder`, which specifies the directory where data files are located (set to `./data/`); `img_dim1` and `img_dim2`, representing the dimensions of images used for plots, set to 20 and 10 respectively; and `SEED`, a random seed value set to 42 for reproducibility.

Following this, the code adjusts the plotting style using `matplotlib`. It sets the default plot style to "seaborn-v0_8" and configures the figure size based on the dimensions defined in the `CFG` class—specifically, width as `img_dim1` and height as `img_dim2`.

Finally, it seeds NumPy’s random number generator with the value stored in `CFG.SEED`. This ensures that any operations involving randomness will produce consistent results each time the code is run, aiding in reproducibility of experiments or analyses.

# Utils

In [3]:
def forecast_metrics(y_true, y_pred):
    return {
        "MAE": float(round(mean_absolute_error(y_true, y_pred), 2)),
        "RMSE": float(round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)),
    }

This code defines a function called `forecast_metrics` that calculates and returns common error metrics used to evaluate the accuracy of time series forecasts.

The function takes two arguments: `y_true`, representing the actual observed values, and `y_pred`, representing the predicted values from a forecasting model. 

Inside the function, it computes the Mean Absolute Error (MAE) using `mean_absolute_error` from scikit-learn and the Root Mean Squared Error (RMSE) by taking the square root of the result from `mean_squared_error` (also from scikit-learn). Both results are rounded to two decimal places and converted to floating-point numbers.

The function then returns a dictionary containing these two metrics, labeled as "MAE" and "RMSE", providing a concise summary of forecast performance.

In [38]:
def merge_forecasts(y_test_df, y_pred_df):
    return pd.merge(y_test_df, y_pred_df, on=['ds', 'unique_id'])

This code defines a function named `merge_forecasts` that combines actual and predicted time series data into a single DataFrame.

The function accepts two arguments: `y_test_df`, which is a pandas DataFrame containing the true (observed) values, and `y_pred_df`, a DataFrame holding the forecasted values. 

It uses the `pd.merge` function from the pandas library to join these two DataFrames based on common columns. The `on=['ds', 'unique_id']` argument specifies that the merge should be performed using the ‘ds’ ( representing date/timestamp) and ‘unique_id’ columns, which are assumed to uniquely identify each time series within the data. 

The function returns a new DataFrame where rows from `y_test_df` and `y_pred_df` are combined based on matching values in these specified columns, allowing for direct comparison of actual versus predicted values.

In [35]:
def compute_overall_error(df, actual_col, pred_col, metric='rmse'):
    return forecast_metrics(df[actual_col], df[pred_col], metric=metric)

TODO

# Baseline

In [ ]:
# M5 competition data - aggregated and prepared in https://github.com/tng-konrad/time-series-with-Konrad/blob/main/helper-m04.ipynb
df = pd.read_csv(CFG.data_folder + 'm04_sales_data_prepared.csv')
# add a (dummy) column for total level
df['top_level'] = 'Total'
# hierarchy structure: mid level: middle_level, bottom level: bottom_level
df.rename(columns={'state_id': 'middle_level', 'store_id': 'bottom_level'}, inplace=True)
# arrange columns
df = df[['ds', 'top_level',  'middle_level', 'bottom_level', 'y']]
# format date column
df['ds'] = pd.to_datetime(df['ds'])

TODO

In [39]:
# prepare hierarchical data
hierarchy_levels = [
    ["top_level"],  ["top_level", "middle_level"],  ["top_level", "middle_level", "bottom_level"] ]

Y_hier_df, S_df, tags = aggregate(df=df, spec=hierarchy_levels)

print("S_df.shape", S_df.shape)
print("Y_hier_df.shape", Y_hier_df.shape)

# quick check
for k in tags.keys():
    print('---')
    print(f"tags['{k}'] :", tags[k])
    print(f"tags['{k}'] shape:", tags[k].shape)



S_df.shape (14, 11)
Y_hier_df.shape (12222, 3)
---
tags['top_level'] : ['Total']
tags['top_level'] shape: (1,)
---
tags['top_level/middle_level'] : ['Total/CA' 'Total/TX' 'Total/WI']
tags['top_level/middle_level'] shape: (3,)
---
tags['top_level/middle_level/bottom_level'] : ['Total/CA/CA_1' 'Total/CA/CA_2' 'Total/CA/CA_3' 'Total/CA/CA_4'
 'Total/TX/TX_1' 'Total/TX/TX_2' 'Total/TX/TX_3' 'Total/WI/WI_1'
 'Total/WI/WI_2' 'Total/WI/WI_3']
tags['top_level/middle_level/bottom_level'] shape: (10,)


TODO

In [40]:
horizon = 7 

Y_test_df = Y_hier_df.groupby("unique_id", as_index=False).tail(horizon)
Y_train_df = Y_hier_df.drop(Y_test_df.index)

TODO

In [41]:
# Compute base auto-ARIMA predictions
model = AutoARIMA(season_length=7)
fcst = StatsForecast(models=[model], freq="D", n_jobs=-1)
Y_hat_df = fcst.forecast(df=Y_train_df, h=4, fitted=True)
Y_fitted_df = fcst.forecast_fitted_values()

TODO

In [ ]:
xmat = merge_forecasts(Y_test_df, Y_hat_df)
# keep track of the prediction column name
string1 = xmat.columns[-1]

xmat.head(3)

,unique_id,ds,y,AutoARIMA
0,Total,2016-05-16,12,14.092568
1,Total,2016-05-17,11,14.102201
2,Total,2016-05-18,22,14.102201


In [ ]:
overall_rmse = forecast_metrics(xmat['y'], xmat[string1])
print(f'Overall RMSE: {overall_rmse}')


for k, ids in tags.items():
    mask = xmat['unique_id'].isin(ids)
    k_metrics = forecast_metrics(xmat.loc[mask, 'y'], xmat.loc[mask, string1])
    print(f"{k} RMSE: {k_metrics}")

Overall RMSE: {'MAE': 1.54, 'RMSE': 2.15}
top_level RMSE: {'MAE': 4.55, 'RMSE': 5.06}
top_level/middle_level RMSE: {'MAE': 1.97, 'RMSE': 2.45}
top_level/middle_level/bottom_level RMSE: {'MAE': 1.12, 'RMSE': 1.45}


TODO

# BottomUp

In [11]:
reconcilers = [BottomUp()] 
hrec = HierarchicalReconciliation(reconcilers=reconcilers)

Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_fitted_df, S_df=S_df, tags=tags)
Y_rec_df.head(3)

,unique_id,ds,AutoARIMA,AutoARIMA/BottomUp
0,Total,2016-05-16,14.092568,13.943087
1,Total,2016-05-17,14.102201,13.327107
2,Total,2016-05-18,14.102201,13.409471


In [12]:
# Merge reconciled forecasts with test data
xmat_rec1 = pd.merge(left=Y_test_df, right=Y_rec_df, on=['ds', 'unique_id'])
xmat_rec1.head(3)


,unique_id,ds,y,AutoARIMA,AutoARIMA/BottomUp
0,Total,2016-05-16,12,14.092568,13.943087
1,Total,2016-05-17,11,14.102201,13.327107
2,Total,2016-05-18,22,14.102201,13.409471


TODO

In [13]:
overall_rmse = forecast_metrics(xmat_rec1['y'], xmat_rec1['AutoARIMA/BottomUp'])
print(f'Overall RMSE: {overall_rmse}')


for k, ids in tags.items():
    mask = xmat_rec1['unique_id'].isin(ids)
    k_metrics = forecast_metrics(xmat_rec1.loc[mask, 'y'], xmat_rec1.loc[mask, 'AutoARIMA/BottomUp'])
    print(f"{k} RMSE: {k_metrics}")

Overall RMSE: {'MAE': 1.54, 'RMSE': 2.18}
top_level RMSE: {'MAE': 4.4, 'RMSE': 5.14}
top_level/middle_level RMSE: {'MAE': 2.01, 'RMSE': 2.52}
top_level/middle_level/bottom_level RMSE: {'MAE': 1.12, 'RMSE': 1.45}


In [14]:
print("Overall comparison:")
baseline_rmse = forecast_metrics(xmat_rec1['y'], xmat_rec1['AutoARIMA'])
reconciled_rmse = forecast_metrics(xmat_rec1['y'], xmat_rec1['AutoARIMA/BottomUp'])
print(f"Baseline RMSE: {baseline_rmse}")
print(f"Reconciled RMSE: {reconciled_rmse}")
# print(f"Delta RMSE (Reconciled - Baseline): {reconciled_rmse - baseline_rmse}")

print("\nPer-level comparison:")
comparison = []

for k, ids in tags.items():
    mask = xmat_rec1['unique_id'].isin(ids)

    baseline_err = forecast_metrics(
        xmat_rec1.loc[mask, 'y'],
        xmat_rec1.loc[mask, 'AutoARIMA']
    )

    reconciled_err = forecast_metrics(
        xmat_rec1.loc[mask, 'y'],
        xmat_rec1.loc[mask, 'AutoARIMA/BottomUp']
    )

    # delta = reconciled_err - baseline_err

    # print(f"{k} — Baseline: {baseline_err:.4f}, Reconciled: {reconciled_err:.4f}, Delta: {delta:.4f}")

    comparison.append({
        'level': k,
        'baseline_rmse': baseline_err,
        'reconciled_rmse': reconciled_err,
        # 'delta_rmse': delta,
#         'improved': reconciled_err < baseline_err
    })

comparison_df = pd.DataFrame(comparison)
comparison_df


Overall comparison:
Baseline RMSE: {'MAE': 1.54, 'RMSE': 2.15}
Reconciled RMSE: {'MAE': 1.54, 'RMSE': 2.18}

Per-level comparison:


,level,baseline_rmse,reconciled_rmse
0,top_level,"{'MAE': 4.55, 'RMSE': 5.06}","{'MAE': 4.4, 'RMSE': 5.14}"
1,top_level/middle_level,"{'MAE': 1.97, 'RMSE': 2.45}","{'MAE': 2.01, 'RMSE': 2.52}"
2,top_level/middle_level/bottom_level,"{'MAE': 1.12, 'RMSE': 1.45}","{'MAE': 1.12, 'RMSE': 1.45}"


TODO

# MiddleOut

In [20]:
reconcilers = [
    MiddleOut(middle_level = 'top_level/middle_level', top_down_method='forecast_proportions')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)

Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_fitted_df, S_df=S_df, tags=tags)
Y_rec_df.head(3)

,unique_id,ds,AutoARIMA,AutoARIMA/MiddleOut_middle_level-top_level/middle_level_top_down_method-forecast_proportions
0,Total,2016-05-16,14.092568,13.929545
1,Total,2016-05-17,14.102201,13.836553
2,Total,2016-05-18,14.102201,13.702898


In [ ]:
# Merge reconciled forecasts with test data
xmat_rec2 = merge_forecasts(Y_test_df, Y_rec_df)
xmat_rec2.head(3)


,unique_id,ds,y,AutoARIMA,AutoARIMA/MiddleOut_middle_level-top_level/middle_level_top_down_method-forecast_proportions
0,Total,2016-05-16,12,14.092568,13.929545
1,Total,2016-05-17,11,14.102201,13.836553
2,Total,2016-05-18,22,14.102201,13.702898


In [24]:
overall_rmse = forecast_metrics(xmat_rec2['y'], xmat_rec2['AutoARIMA/MiddleOut_middle_level-top_level/middle_level_top_down_method-forecast_proportions'])
print(f'Overall RMSE: {overall_rmse}')


for k, ids in tags.items():
    mask = xmat_rec2['unique_id'].isin(ids)
    k_metrics = forecast_metrics(xmat_rec2.loc[mask, 'y'], xmat_rec2.loc[mask, 'AutoARIMA/MiddleOut_middle_level-top_level/middle_level_top_down_method-forecast_proportions'])
    print(f"{k} RMSE: {k_metrics}")

Overall RMSE: {'MAE': 1.54, 'RMSE': 2.15}
top_level RMSE: {'MAE': 4.48, 'RMSE': 5.11}
top_level/middle_level RMSE: {'MAE': 1.97, 'RMSE': 2.45}
top_level/middle_level/bottom_level RMSE: {'MAE': 1.12, 'RMSE': 1.44}


In [26]:
xstring = 'AutoARIMA/MiddleOut_middle_level-top_level/middle_level_top_down_method-forecast_proportions'

print("Overall comparison:")
baseline_rmse = forecast_metrics(xmat_rec2['y'], xmat_rec2['AutoARIMA'])
reconciled_rmse = forecast_metrics(xmat_rec2['y'], xmat_rec2[xstring])
print(f"Baseline RMSE: {baseline_rmse}")
print(f"Reconciled RMSE: {reconciled_rmse}")
# print(f"Delta RMSE (Reconciled - Baseline): {reconciled_rmse - baseline_rmse}")

print("\nPer-level comparison:")
comparison = []

for k, ids in tags.items():
    mask = xmat_rec2['unique_id'].isin(ids)

    baseline_err = forecast_metrics(
        xmat_rec2.loc[mask, 'y'],
        xmat_rec2.loc[mask, 'AutoARIMA']
    )

    reconciled_err = forecast_metrics(
        xmat_rec2.loc[mask, 'y'],
        xmat_rec2.loc[mask, xstring]
    )

    # delta = reconciled_err - baseline_err

    # print(f"{k} — Baseline: {baseline_err:.4f}, Reconciled: {reconciled_err:.4f}, Delta: {delta:.4f}")

    comparison.append({
        'level': k,
        'baseline_rmse': baseline_err,
        'reconciled_rmse': reconciled_err,
        # 'delta_rmse': delta,
#         'improved': reconciled_err < baseline_err
    })

comparison_df = pd.DataFrame(comparison)
comparison_df


Overall comparison:
Baseline RMSE: {'MAE': 1.54, 'RMSE': 2.15}
Reconciled RMSE: {'MAE': 1.54, 'RMSE': 2.15}

Per-level comparison:


,level,baseline_rmse,reconciled_rmse
0,top_level,"{'MAE': 4.55, 'RMSE': 5.06}","{'MAE': 4.48, 'RMSE': 5.11}"
1,top_level/middle_level,"{'MAE': 1.97, 'RMSE': 2.45}","{'MAE': 1.97, 'RMSE': 2.45}"
2,top_level/middle_level/bottom_level,"{'MAE': 1.12, 'RMSE': 1.45}","{'MAE': 1.12, 'RMSE': 1.44}"


# TopDown

In [27]:
reconcilers = [
   TopDown(method='forecast_proportions')]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)

Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_fitted_df, S_df=S_df, tags=tags)
Y_rec_df.head(3)

,unique_id,ds,AutoARIMA,AutoARIMA/TopDown_method-forecast_proportions
0,Total,2016-05-16,14.092568,14.092568
1,Total,2016-05-17,14.102201,14.102201
2,Total,2016-05-18,14.102201,14.102201


In [ ]:
xmat_rec3 = merge_forecasts(Y_test_df, Y_rec_df)


,unique_id,ds,y,AutoARIMA,AutoARIMA/TopDown_method-forecast_proportions
0,Total,2016-05-16,12,14.092568,14.092568
1,Total,2016-05-17,11,14.102201,14.102201
2,Total,2016-05-18,22,14.102201,14.102201


In [29]:
xstring = "AutoARIMA/TopDown_method-forecast_proportions"

overall_rmse = forecast_metrics(xmat_rec3['y'], xmat_rec3[xstring])
print(f'Overall RMSE: {overall_rmse}')


for k, ids in tags.items():
    mask = xmat_rec3['unique_id'].isin(ids)
    k_metrics = forecast_metrics(xmat_rec3.loc[mask, 'y'], xmat_rec3.loc[mask, xstring])
    print(f"{k} RMSE: {k_metrics}")

Overall RMSE: {'MAE': 1.55, 'RMSE': 2.14}
top_level RMSE: {'MAE': 4.55, 'RMSE': 5.06}
top_level/middle_level RMSE: {'MAE': 1.96, 'RMSE': 2.43}
top_level/middle_level/bottom_level RMSE: {'MAE': 1.12, 'RMSE': 1.44}


In [30]:

print("Overall comparison:")
baseline_rmse = forecast_metrics(xmat_rec3['y'], xmat_rec3['AutoARIMA'])
reconciled_rmse = forecast_metrics(xmat_rec3['y'], xmat_rec3[xstring])
print(f"Baseline RMSE: {baseline_rmse}")
print(f"Reconciled RMSE: {reconciled_rmse}")
# print(f"Delta RMSE (Reconciled - Baseline): {reconciled_rmse - baseline_rmse}")

print("\nPer-level comparison:")
comparison = []

for k, ids in tags.items():
    mask = xmat_rec3['unique_id'].isin(ids)

    baseline_err = forecast_metrics(
        xmat_rec3.loc[mask, 'y'],
        xmat_rec3.loc[mask, 'AutoARIMA']
    )

    reconciled_err = forecast_metrics(
        xmat_rec3.loc[mask, 'y'],
        xmat_rec3.loc[mask, xstring]
    )

    # delta = reconciled_err - baseline_err

    # print(f"{k} — Baseline: {baseline_err:.4f}, Reconciled: {reconciled_err:.4f}, Delta: {delta:.4f}")

    comparison.append({
        'level': k,
        'baseline_rmse': baseline_err,
        'reconciled_rmse': reconciled_err,
        # 'delta_rmse': delta,
#         'improved': reconciled_err < baseline_err
    })

comparison_df = pd.DataFrame(comparison)
comparison_df


Overall comparison:
Baseline RMSE: {'MAE': 1.54, 'RMSE': 2.15}
Reconciled RMSE: {'MAE': 1.55, 'RMSE': 2.14}

Per-level comparison:


,level,baseline_rmse,reconciled_rmse
0,top_level,"{'MAE': 4.55, 'RMSE': 5.06}","{'MAE': 4.55, 'RMSE': 5.06}"
1,top_level/middle_level,"{'MAE': 1.97, 'RMSE': 2.45}","{'MAE': 1.96, 'RMSE': 2.43}"
2,top_level/middle_level/bottom_level,"{'MAE': 1.12, 'RMSE': 1.45}","{'MAE': 1.12, 'RMSE': 1.44}"


# MinTrace

In [31]:
reconcilers = [
    MinTrace(method='ols'),
]

hrec = HierarchicalReconciliation(reconcilers=reconcilers)

Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_fitted_df, S_df=S_df, tags=tags)
Y_rec_df.head(3)



,unique_id,ds,AutoARIMA,AutoARIMA/MinTrace_method-ols
0,Total,2016-05-16,14.092568,14.050535
1,Total,2016-05-17,14.102201,13.992761
2,Total,2016-05-18,14.102201,13.970089


In [ ]:

xmat_rec4 = merge_forecasts(Y_test_df, Y_rec_df)
xmat_rec4.head(3)

,unique_id,ds,y,AutoARIMA,AutoARIMA/MinTrace_method-ols
0,Total,2016-05-16,12,14.092568,14.050535
1,Total,2016-05-17,11,14.102201,13.992761
2,Total,2016-05-18,22,14.102201,13.970089


In [33]:
xstring = "AutoARIMA/MinTrace_method-ols"

overall_rmse = forecast_metrics(xmat_rec4['y'], xmat_rec4[xstring])
print(f'Overall RMSE: {overall_rmse}')


for k, ids in tags.items():
    mask = xmat_rec4['unique_id'].isin(ids)
    k_metrics = forecast_metrics(xmat_rec4.loc[mask, 'y'], xmat_rec4.loc[mask, xstring])
    print(f"{k} RMSE: {k_metrics}")

Overall RMSE: {'MAE': 1.55, 'RMSE': 2.15}
top_level RMSE: {'MAE': 4.53, 'RMSE': 5.07}
top_level/middle_level RMSE: {'MAE': 1.99, 'RMSE': 2.45}
top_level/middle_level/bottom_level RMSE: {'MAE': 1.12, 'RMSE': 1.44}


In [34]:

print("Overall comparison:")
baseline_rmse = forecast_metrics(xmat_rec4['y'], xmat_rec4['AutoARIMA'])
reconciled_rmse = forecast_metrics(xmat_rec4['y'], xmat_rec4[xstring])
print(f"Baseline RMSE: {baseline_rmse}")
print(f"Reconciled RMSE: {reconciled_rmse}")
# print(f"Delta RMSE (Reconciled - Baseline): {reconciled_rmse - baseline_rmse}")

print("\nPer-level comparison:")
comparison = []

for k, ids in tags.items():
    mask = xmat_rec4['unique_id'].isin(ids)

    baseline_err = forecast_metrics(
        xmat_rec4.loc[mask, 'y'],
        xmat_rec4.loc[mask, 'AutoARIMA']
    )

    reconciled_err = forecast_metrics(
        xmat_rec4.loc[mask, 'y'],
        xmat_rec4.loc[mask, xstring]
    )

    # delta = reconciled_err - baseline_err

    # print(f"{k} — Baseline: {baseline_err:.4f}, Reconciled: {reconciled_err:.4f}, Delta: {delta:.4f}")

    comparison.append({
        'level': k,
        'baseline_rmse': baseline_err,
        'reconciled_rmse': reconciled_err,
        # 'delta_rmse': delta,
#         'improved': reconciled_err < baseline_err
    })

comparison_df = pd.DataFrame(comparison)
comparison_df


Overall comparison:
Baseline RMSE: {'MAE': 1.54, 'RMSE': 2.15}
Reconciled RMSE: {'MAE': 1.55, 'RMSE': 2.15}

Per-level comparison:


,level,baseline_rmse,reconciled_rmse
0,top_level,"{'MAE': 4.55, 'RMSE': 5.06}","{'MAE': 4.53, 'RMSE': 5.07}"
1,top_level/middle_level,"{'MAE': 1.97, 'RMSE': 2.45}","{'MAE': 1.99, 'RMSE': 2.45}"
2,top_level/middle_level/bottom_level,"{'MAE': 1.12, 'RMSE': 1.45}","{'MAE': 1.12, 'RMSE': 1.44}"
